In [39]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn import linear_model 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.svm import SVC
from sklearn. preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.datasets import load_wine
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_digits
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import ShuffleSplit
import tensorflow as tf
import os

LOAD DATA and PREPROCCESS

In [40]:
df = pd.read_csv('train.csv')
df.count()
# df.head()

id               159571
comment_text     159571
toxic            159571
severe_toxic     159571
obscene          159571
threat           159571
insult           159571
identity_hate    159571
dtype: int64

In [41]:
df[df['toxic']==1]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
...,...,...,...,...,...,...,...,...
159494,fef4cf7ba0012866,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1
159514,ff39a2895fc3b40e,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0
159541,ffa33d3122b599d6,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0
159546,ffb47123b2d82762,"""\n\nHey listen don't you ever!!!! Delete my e...",1,0,0,0,1,0


Peprocess

In [42]:
from tensorflow.keras.layers import TextVectorization

In [43]:
X= df['comment_text'].values #values zamienia na tablice numpy



In [44]:
Y = df.drop(['comment_text','id'],axis=1).values
Y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [45]:
MAX_FETURES = 200000 #words in vocabulary

In [46]:
vectorizer = TextVectorization(max_tokens=MAX_FETURES,output_sequence_length=130,output_mode='int')

In [47]:
vectorizer.adapt(X)

In [48]:
vectorizer('hello, good morning')[0:3]

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([ 288,   98, 2263], dtype=int64)>

In [49]:
vectorized_text = vectorizer(X)

In [50]:
vectorized_text# zamienia nasza tablice slow na tekonizator

<tf.Tensor: shape=(159571, 130), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [51]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,Y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [52]:
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))
len(test)

997

In [53]:
train_generator = train.as_numpy_iterator()
train_generator.next()

(array([[ 1787,     8,   988, ...,     0,     0,     0],
        [93409,   265,    27, ...,     0,     0,     0],
        [   76,    19,     7, ...,     0,     0,     0],
        ...,
        [  712,   116,     7, ...,     0,     0,     0],
        [  298,    18,   666, ...,     0,     0,     0],
        [17804,    21,   216, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [1, 0, 1, 1, 1, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

CREATING MODEL

In [54]:
from tensorflow.keras.layers import Dense,Dropout,Bidirectional,Embedding, LSTM
from tensorflow.keras.models import Sequential

In [57]:
model = Sequential()
model.add(Embedding(MAX_FETURES+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))#pozwala nam dzialac wstecz, przydatne bo wystepuja slowa jak dont

model.add(Dense(128,activation ='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))

model.add(Dense(6,activation='sigmoid'))

In [58]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam',metrics = ['accuracy'])

In [59]:
history = model.fit(train,epochs=5,validation_data=val)

Epoch 1/5
6981/6981 [==============================] - 434s 62ms/step - loss: 0.0628 - accuracy: 0.9755 - val_loss: 0.0459 - val_accuracy: 0.9946
Epoch 2/5
6981/6981 [==============================] - 426s 61ms/step - loss: 0.0459 - accuracy: 0.9820 - val_loss: 0.0402 - val_accuracy: 0.9939
Epoch 3/5
6981/6981 [==============================] - 428s 61ms/step - loss: 0.0406 - accuracy: 0.9906 - val_loss: 0.0360 - val_accuracy: 0.9942
Epoch 4/5
6981/6981 [==============================] - 437s 63ms/step - loss: 0.0364 - accuracy: 0.9777 - val_loss: 0.0329 - val_accuracy: 0.9940
Epoch 5/5
6981/6981 [==============================] - 427s 61ms/step - loss: 0.0325 - accuracy: 0.9755 - val_loss: 0.0294 - val_accuracy: 0.9938


In [60]:
batch = test.as_numpy_iterator().next()

In [114]:
input_text = vectorizer(" kfc have shitty food")
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [115]:
results = model.predict(np.expand_dims(input_text,0))
results

1/1 [==============================] - 0s 19ms/step


array([[0.73908186, 0.01182667, 0.2757948 , 0.01827754, 0.31640902,
        0.04107217]], dtype=float32)

In [116]:
res_df = pd.DataFrame(results)
labels_df = pd.DataFrame(df.columns[2:])
df_merged = pd.concat([res_df,labels_df],axis=1)
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
